In [ ]:
import pandas as pd
import gzip
import json
import numpy as np
import os

# Function to calculate helpfulness score (hij)
def calculate_helpfulness_score(user_reviews):
    # Group reviews by user and item
    result = {}
    
    for review in user_reviews:
        user_id = review.get('reviewerID')
        item_id = review.get('asin')
        helpful = review.get('helpful', [0, 0])
        
        # Extract helpful votes and total votes
        helpful_votes = helpful[0]
        total_votes = helpful[1]
        
        # Avoid division by zero
        if total_votes == 0:
            total_votes = 1
            
        # Calculate bij = (# helpful votes)² / # total votes
        bij = (helpful_votes ** 2) / total_votes
        
        # Store bij values to later calculate hij
        if item_id not in result:
            result[item_id] = {}
        
        result[item_id][user_id] = {'bij': bij, 'review': review}
    
    # Calculate hij = bij / ∑bxj for each user-item pair
    for item_id, users in result.items():
        sum_bxj = sum(user_data['bij'] for user_data in users.values())
        
        # Avoid division by zero
        if sum_bxj == 0:
            sum_bxj = 1
            
        for user_id in users:
            users[user_id]['hij'] = users[user_id]['bij'] / sum_bxj
    
    return result

# Function to calculate reliability scores (rhij)
def calculate_reliability_scores(helpfulness_data, alpha=0.5):
    for item_id, users in helpfulness_data.items():
        # Convert users to a list for indexing
        user_list = list(users.keys())
        n_prime = len(user_list)
        
        # Calculate zij and qij for each user
        for i, user_id in enumerate(user_list):
            # Calculate zij for most recent review reliability
            zij = sum(1/(e**2) for e in range(1, n_prime-i+1))
            
            # For simplicity, using a constant for σ²ᵢⱼ
            sigma_squared = 1.0
            
            # Calculate qij for top ranking reviewer reliability
            qij = (1/sigma_squared) * (n_prime - i)
            
            users[user_id]['zij'] = zij
            users[user_id]['qij'] = qij
        
        # Calculate mostij and topij
        sum_zxj = sum(users[user_id]['zij'] for user_id in user_list)
        sum_qxj = sum(users[user_id]['qij'] for user_id in user_list)
        
        # Avoid division by zero
        if sum_zxj == 0:
            sum_zxj = 1
        if sum_qxj == 0:
            sum_qxj = 1
            
        for user_id in user_list:
            users[user_id]['mostij'] = users[user_id]['zij'] / sum_zxj
            users[user_id]['topij'] = users[user_id]['qij'] / sum_qxj
            
            # Calculate combined reliability score (dij)
            dij = alpha * users[user_id]['topij'] + (1-alpha) * users[user_id]['mostij']
            
            # For this implementation, we'll use dij as rhij
            users[user_id]['rhij'] = dij
            
            # Calculate average of hij and rhij
            users[user_id]['avg_score'] = (users[user_id]['hij'] + users[user_id]['rhij']) / 2
    
    return helpfulness_data

# Function to generate the CSV file
def generate_csv(processed_data, output_path):
    # Prepare data for CSV
    csv_data = []
    
    for item_id, users in processed_data.items():
        for user_id, scores in users.items():
            csv_data.append({
                'user_id': user_id,
                'item_id': item_id,
                'avg_of_rhij_and_hij': scores['avg_score']
            })
    
    # Create DataFrame and save as CSV
    df = pd.DataFrame(csv_data)
    df.to_csv(output_path, index=False)
    
    return df

# Main function to process the data
def process_review_data(file_path, output_path):
    # Load the JSON data from the gzip file
    data = []
    
    try:
        with gzip.open(file_path, 'rt', encoding='utf-8') as f:
            for line in f:
                data.append(json.loads(line))
        
        # Calculate helpfulness scores
        helpfulness_data = calculate_helpfulness_score(data)
        
        # Calculate reliability scores and average
        processed_data = calculate_reliability_scores(helpfulness_data)
        
        # Generate and save CSV
        df = generate_csv(processed_data, output_path)
        
        print(f"CSV file successfully generated at {output_path}")
        print(f"Sample of the generated data:")
        print(df.head())
        
    except FileNotFoundError:
        print(f"Error: The file {file_path} was not found.")
    except Exception as e:
        print(f"Error processing the data: {str(e)}")

# File paths
file_path = "C:\\Users\\KIIT\\Downloads\\Appliances.json.gz"
output_path = "C:\\Users\\KIIT\\Downloads\\average_scores1.csv"

# Process the data
process_review_data(file_path, output_path)


In [2]:
import pandas as pd
import numpy as np
import os
from scipy.sparse.linalg import svds
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
import tempfile
import subprocess

# ---------- STEP 1: Parse Arts.txt ----------
def parse_arts_file(file_path):
    reviews = []
    current_review = {}
    with open(file_path, 'r', encoding='utf-8') as f:
        for line in f:
            line = line.strip()
            if not line:
                continue
            if line.startswith('product/productId:'):
                if current_review:
                    reviews.append(current_review)
                    current_review = {}
                current_review['product/productId'] = line.split(': ')[1]
            elif line.startswith('review/userId:'):
                current_review['review/userId'] = line.split(': ')[1]
            elif line.startswith('review/helpfulness:'):
                helpful = line.split(': ')[1].split('/')
                try:
                    current_review['review/helpfulness'] = [int(helpful[0]), int(helpful[1])]
                except:
                    current_review['review/helpfulness'] = [0, 1]
    if current_review:
        reviews.append(current_review)
    return reviews

# ---------- STEP 2: Helpfulness and Reliability ----------
def calculate_helpfulness_score(reviews):
    result = {}
    for review in reviews:
        user_id = review.get('review/userId')
        item_id = review.get('product/productId')
        helpful = review.get('review/helpfulness', [0, 0])
        helpful_votes = helpful[0]
        total_votes = helpful[1] if helpful[1] != 0 else 1
        bij = (helpful_votes ** 2) / total_votes
        if item_id not in result:
            result[item_id] = {}
        result[item_id][user_id] = {'bij': bij, 'review': review}
    for item_id, users in result.items():
        sum_bxj = sum(user['bij'] for user in users.values()) or 1
        for user_id in users:
            users[user_id]['hij'] = users[user_id]['bij'] / sum_bxj
    return result

def calculate_reliability_scores(helpfulness_data, alpha=0.5):
    for item_id, users in helpfulness_data.items():
        user_list = list(users.keys())
        n_prime = len(user_list)
        for i, user_id in enumerate(user_list):
            zij = sum(1 / (np.e**2) for _ in range(1, max(n_prime - i + 1, 2)))
            sigma_squared = 1.0
            qij = (1 / sigma_squared) * (n_prime - i)
            users[user_id]['zij'] = zij
            users[user_id]['qij'] = qij
        sum_zxj = sum(users[u]['zij'] for u in user_list)
        sum_qxj = sum(users[u]['qij'] for u in user_list)
        for user_id in user_list:
            users[user_id]['mostij'] = users[user_id]['zij'] / sum_zxj
            users[user_id]['topij'] = users[user_id]['qij'] / sum_qxj
            users[user_id]['rhij'] = alpha * users[user_id]['topij'] + (1 - alpha) * users[user_id]['mostij']
            users[user_id]['avg_score'] = (users[user_id]['hij'] + users[user_id]['rhij']) / 2
    return helpfulness_data

# ---------- STEP 3: Generate DataFrame ----------
def generate_dataframe(processed_data):
    rows = []
    for item_id, users in processed_data.items():
        for user_id, data in users.items():
            rows.append({
                'user_id': user_id,
                'item_id': item_id,
                'avg_of_rhij_and_hij': data['avg_score']
            })
    return pd.DataFrame(rows)

# ---------- STEP 4: Matrix Factorization ----------
def apply_svd(df, k=50):
    pivot_df = df.pivot(index='user_id', columns='item_id', values='avg_of_rhij_and_hij').fillna(0)
    R = pivot_df.values
    U, sigma, Vt = svds(R, k=min(k, min(R.shape)-1))
    sigma = np.diag(sigma)
    predicted_ratings = np.dot(np.dot(U, sigma), Vt)
    return pivot_df, predicted_ratings, U, Vt

# ---------- STEP 5: Evaluation ----------
def compute_loss(R_true, R_pred, U, Vt, lambda_reg=0.1):
    mask = R_true > 0
    mse_loss = np.sum((R_true[mask] - R_pred[mask]) ** 2)
    reg_term = lambda_reg * (np.sum(U**2) + np.sum(Vt**2))
    return mse_loss + reg_term

def hit_ratio_at_k(test_R, predicted_R, k=10):
    hits = 0
    total_users = 0
    for user_idx in range(test_R.shape[0]):
        true_items = np.where(test_R[user_idx] > 0)[0]
        if len(true_items) == 0:
            continue
        top_k_items = np.argsort(predicted_R[user_idx])[::-1][:k]
        if np.intersect1d(true_items, top_k_items).size > 0:
            hits += 1
        total_users += 1
    return hits / total_users if total_users else 0

# ---------- STEP 6: Main ----------
def main():
    # Create safe directory in user's home folder
    safe_dir = os.path.join(os.path.expanduser("~"), "RecommendationData")
    os.makedirs(safe_dir, exist_ok=True)
    input_path = os.path.join(safe_dir, "Arts.txt")

    if not os.path.exists(input_path):
        print(f" Please copy Arts.txt to: {safe_dir}")
        print("1. Right-click your Arts.txt file")
        print("2. Select 'Copy'")
        print("3. Open the folder that just appeared")
        print("4. Right-click → Paste")
        os.startfile(safe_dir)
        return

    try:
        print(f" Using file at: {input_path}")
        reviews = parse_arts_file(input_path)
        helpfulness_data = calculate_helpfulness_score(reviews)
        processed_data = calculate_reliability_scores(helpfulness_data)
        df = generate_dataframe(processed_data)

        # Save CSV to safe directory
        output_path = os.path.join(safe_dir, "average_scores12.csv")
        df.to_csv(output_path, index=False)
        print(f"\n CSV saved to: {output_path}")

        print("\n Sample of dataframe:")
        print(df.head())

        # Apply SVD
        pivot_df, predicted_ratings, U, Vt = apply_svd(df, k=50)
        R = pivot_df.values

        # Print matrices (first 5x5)
        print("\n Original Rating Matrix (First 5x5):")
        print(pd.DataFrame(R[:5, :5], index=pivot_df.index[:5], columns=pivot_df.columns[:5]))

        print("\n Predicted Rating Matrix (First 5x5):")
        print(pd.DataFrame(predicted_ratings[:5, :5], index=pivot_df.index[:5], columns=pivot_df.columns[:5]))

        # Loss calculation
        loss = compute_loss(R, predicted_ratings, U, Vt)
        print(f"\n Regularized Loss: {loss:.4f}")

        # Recommendations
        user_index = 0
        user_id = pivot_df.index[user_index]
        top_preds = pd.Series(predicted_ratings[user_index], index=pivot_df.columns).sort_values(ascending=False).head(10)
        print(f"\n Top 10 recommendations for user {user_id}:\n{top_preds}")

        # Train-test evaluation
        user_ids = pivot_df.index.tolist()
        train_users, test_users = train_test_split(user_ids, test_size=0.2, random_state=42)
        train_matrix = pivot_df.loc[train_users].values
        test_matrix = pivot_df.loc[test_users].values

        U_train, sigma_train, Vt_train = svds(train_matrix, k=50)
        sigma_train = np.diag(sigma_train)
        test_pred = np.dot(np.dot(U_train, sigma_train), Vt_train)[:len(test_users), :]

        mask = test_matrix > 0
        mse = mean_squared_error(test_matrix[mask], test_pred[mask])
        hit_ratio = hit_ratio_at_k(test_matrix, test_pred, k=10)

        print(f"\n MSE on test set: {mse:.4f}")
        print(f" Hit Ratio@10: {hit_ratio:.4f}")

    except PermissionError as pe:
        print(f"\n Critical Permission Error: {pe}")
        print("Run these commands in Admin Command Prompt:")
        print(f'  takeown /f "{safe_dir}" /r /d y')
        print(f'  icacls "{safe_dir}" /grant "%username%":F /t')
        print("Then restart your computer and try again")

    except Exception as e:
        print(f"\n Unexpected Error: {e}")
        print("Common fixes:")
        print("1. Ensure Arts.txt has exactly 8 fields per review")
        print("2. Check file encoding is UTF-8")
        print("3. Reduce SVD k parameter if memory error occurs")

if __name__ == "__main__":
    main()

 Using file at: C:\Users\KIIT\RecommendationData\Arts.txt

 CSV saved to: C:\Users\KIIT\RecommendationData\average_scores12.csv

 Sample of dataframe:
          user_id     item_id  avg_of_rhij_and_hij
0  A1QA985ULVCQOB  B000GKXY4S             0.833333
1   ALCX2ELNHLQA7  B000GKXY4S             0.166667
2  A2M2M4R1KG5WOL  B000140KIW             0.666667
3   ARQAQ6ZYMFPCA  B000140KIW             0.133333
4  A3FPG4LAJ1HOHZ  B000140KIW             0.100000

 Original Rating Matrix (First 5x5):
item_id                048629241X  0618307222  0698109228  0806998075  \
user_id                                                                 
A002211213O7OZD4NB6T4         0.0         0.0         0.0         0.0   
A00890531BOEEDK31WCBE         0.0         0.0         0.0         0.0   
A01676021JTRZ0XE5YS4A         0.0         0.0         0.0         0.0   
A03007643BIO3UI6ZO6ZR         0.0         0.0         0.0         0.0   
A03043323VAO4JWT2G7HF         0.0         0.0         0.0         0